# Simple Arrays

In this example, we will show how to start building a simple Array API.

The important things to note are:

1. Our "optimization" replacement isn't tied to any particular type of array. It is agnostic to how we actually represent arrays (in this case strings).
2. We end up with a Python class that acts like an array, but builds up an expression graph. We use this same class to build up the replacements, so they are very succinct and readble.
3. There is no metaprogramming in this example, and no inspecting of type signatures.
4. Everthing is well typed and MyPy will warn us if we are trying to call operations on arrays which are not valid.

We only define two operations, add and getitem:

In [1]:
import metadsl
import typing

In [2]:
class Array(metadsl.Instance):
    @metadsl.call(lambda self, other: Array)
    def __add__(self, other: "Array") -> "Array":
        ...
    
    @metadsl.call(lambda self, idx: Array)
    def __getitem__(self, idx: int) -> "Array":
        ...



We define one replacement, that distributes indexing over addition:

In [6]:
array_replacements = metadsl.Replacements()

@array_replacements.register_pure(Array, Array, None)
def _distribute_add(a: Array, b: Array, c: int):
    return (
        # expression to match agains
        (a + b)[c],
        # expression it is replaced with
        a[c] + b[c]
    )

Now we define a simple "implementation" of arrays, that are just strings representing their names.

We write how to convert adding and indexing into strings, to "compile" the expression:

In [3]:
array_string_replacements = metadsl.Replacements()


all_replacements = metadsl.ReplacementsSequence(array_replacements, array_string_replacements)

@metadsl.call(lambda x: Array)
def array(x: str) -> Array:
    ...


@metadsl.call(lambda x: all_replacements)
def as_string(x: Array) -> str:
    ...

    
@array_string_replacements.register_pure(None)
def _string(x: str):
    return as_string(array(x)), x

@array_string_replacements.register(None, None)
def _add(x: str, y: str):
    return array(x) + array(y), lambda: array(f"({x} + {y})")


@array_string_replacements.register(None, None)
def _getitem(x: str, y: int):
    return array(x)[y], lambda: array(f"{x}[{y}]")

Now we can create two arrays, add them and index, and turn them into a string, which will run the replacments:

In [5]:
a = array("a")
b = array("b")

res = (a + b)[10]
print(as_string(res))

(a[10] + B[10])
